<a href="https://colab.research.google.com/github/srini229/EE5333_tutorials/blob/master/intro/Introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <ins>**Set cover problem**</ins>

## Problem definition:
<ul>
$\mathcal{U}$ : Universe = set of elements, $\{0,1,2,\ldots,N-1\}$

$\mathcal{S}$ : collection of subsets of $\mathcal{U}$

$\mathcal{C}$ : set cover is a subset of $\mathcal{S}$ with smallest cardinality whose union is $\mathcal{U}$
</ul>

## Example:
<ul>
$\mathcal{U}=\{0,1,2,3,4\},~\mathcal{S}=\{\{0,1\},\{1,2,3\},\{3,4\},\{0,1,4\}\}$

$\mathcal{C}=\{\{1,2,3\},\{0,1,4\}\}$
</ul>

# ILP formulation
* $x_s$ is an indicator variable for $s\in \mathcal{S}$ being in $\mathcal{C}$; $\small x_s = \begin{cases}0,& i \notin \mathcal{C}\\1,& i \in \mathcal{C}	\end{cases}$

* Objective:
  $\min\limits_{x_s} \sum\limits_{s\in \mathcal{S}}x_s$

* Subject to constraints:

  $\begin{align} \sum\limits_{s:i\in s}x_s &\geq 1,&\forall i \in \mathcal{U}\\
x_s &\in \{0,1\}, &\forall s \in \mathcal{S}
\end{align}$





In [ ]:
# Install the mip package inside the colab environment
%pip install mip

In [ ]:
# N : number of vertices; vertices in {0,1,...(N-1)}
# S : list of tuples
# Return value : set cover as a list of tuples

def findSetCover(N, S):
    import mip
    model = mip.Model("SetCover")
    model.verbose = 0 # turn off Cbc console logs
    x = [model.add_var(var_type=mip.BINARY,\
        name=f"x_{s}") for s in range(len(S))]
    model.objective = mip.minimize(mip.xsum(x))
    occ = [[] for _ in range(N)]
    for i in range(len(S)):
      for j in S[i]: # store the set indices to which i belongs
        occ[j].append(i)
    for xi in occ:
      model += mip.xsum(x[i] for i in xi) >= 1
    model.optimize()
    model.write('set_cover.lp')
    if model.status == mip.OptimizationStatus.OPTIMAL:
        return [S[i] for i in range(len(S)) if x[i].x > 0.9]
    return []


In [ ]:
print(findSetCover(5, [(0,1), (1,2,3), (3,4), (0,1,4)]))

[(1, 2, 3), (0, 1, 4)]


In [ ]:
!cat set_cover.lp

\Problem name: SetCover

Minimize
OBJROW: x_0 + x_1 + x_2 + x_3
Subject To
constr(0):  x_0 + x_3 >= 1
constr(1):  x_0 + x_1 + x_3 >= 1
constr(2):  x_1 >= 1
constr(3):  x_1 + x_2 >= 1
constr(4):  x_2 + x_3 >= 1
Bounds
 0 <= x_0 <= 1
 0 <= x_1 <= 1
 0 <= x_2 <= 1
 0 <= x_3 <= 1
Integers
x_0 x_1 x_2 x_3 
End
